In [1]:
import pandas as pd
import pickle 

pd.set_option('display.max_columns', None)

In [5]:
from utils import non_weather_training_columns_list

In [56]:
def forecast_next_24_hours_output_flow_rate(
        input_df: pd.DataFrame,  
        model, 
        year: int, 
        month: int, 
        day: int, 
        hour: int, 
        include_weather_features: bool = False
    ) -> pd.DataFrame:
    
    timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
    input_df = input_df[input_df.timestamp == timestamp]
    if include_weather_features:
        input_df = input_df.drop(columns=['timestamp'])
    else:
        input_df = input_df[non_weather_training_columns_list]
    
    targets = [f'target_{i}' for i in range(1, 25)]
    
    X = input_df.drop(columns=targets).values
    print(X.shape)
    y = input_df[targets]
    
    past_outputs = []
    forecasts = [y.values]
    for i in range(1, 25):
        target = y[f'target_{i}']
        next_hour_forecast = model.predict(X)[0]
        
    
    return input_df
    
    

In [57]:
df = pd.read_parquet("../data/silver/training_dataset.parquet")
model = pickle.load(open("../models/xgb_1h.pkl", "rb"))
model_w = pickle.load(open("../models/xgb_with_weather_1h.pkl", "rb"))

In [58]:
df2 = forecast_next_24_hours_output_flow_rate(df, model,  2023, 9, 15, 18, False)

(1, 64)


ValueError: Feature shape mismatch, expected: 30, got 64

In [53]:
df2

,hour,day_of_week,week_of_year,year,input_flow_rate_lag_1,reservoir_level_percentage_lag_1,pressure_lag_1,output_flow_rate_lag_1,pump_1_lag_1,pump_2_lag_1,input_flow_rate_lag_2,reservoir_level_percentage_lag_2,pressure_lag_2,output_flow_rate_lag_2,pump_1_lag_2,pump_2_lag_2,input_flow_rate_lag_3,reservoir_level_percentage_lag_3,pressure_lag_3,output_flow_rate_lag_3,pump_1_lag_3,pump_2_lag_3,input_flow_rate_lag_6,reservoir_level_percentage_lag_6,pressure_lag_6,output_flow_rate_lag_6,pump_1_lag_6,pump_2_lag_6,input_flow_rate_lag_12,reservoir_level_percentage_lag_12,pressure_lag_12,output_flow_rate_lag_12,pump_1_lag_12,pump_2_lag_12,input_flow_rate_lag_24,reservoir_level_percentage_lag_24,pressure_lag_24,output_flow_rate_lag_24,pump_1_lag_24,pump_2_lag_24,input_flow_rate_roll_mean_3,reservoir_level_percentage_roll_mean_3,pressure_roll_mean_3,output_flow_rate_roll_mean_3,pump_1_roll_mean_3,pump_2_roll_mean_3,input_flow_rate_roll_mean_6,reservoir_level_percentage_roll_mean_6,pressure_roll_mean_6,output_flow_rate_roll_mean_6,pump_1_roll_mean_6,pump_2_roll_mean_6,input_flow_rate_roll_mean_12,reservoir_level_percentage_roll_mean_12,pressure_roll_mean_12,output_flow_rate_roll_mean_12,pump_1_roll_mean_12,pump_2_roll_mean_12,input_flow_rate_roll_mean_24,reservoir_level_percentage_roll_mean_24,pressure_roll_mean_24,output_flow_rate_roll_mean_24,pump_1_roll_mean_24,pump_2_roll_mean_24,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24
4350,5,4,37,2023,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,3.16,92.26,32.62,0.0,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,1.316667,66.325,32.468333,1.16375,0.0,0.0,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06


In [54]:
len(df2.columns)

88

In [55]:
len(df2[non_weather_training_columns_list].columns)

88

In [44]:
df2

,input_flow_rate_lag_1,reservoir_level_percentage_lag_1,pressure_lag_1,output_flow_rate_lag_1,pump_1_lag_1,pump_2_lag_1,input_flow_rate_lag_2,reservoir_level_percentage_lag_2,pressure_lag_2,output_flow_rate_lag_2,pump_1_lag_2,pump_2_lag_2,input_flow_rate_lag_3,reservoir_level_percentage_lag_3,pressure_lag_3,output_flow_rate_lag_3,pump_1_lag_3,pump_2_lag_3,input_flow_rate_lag_6,reservoir_level_percentage_lag_6,pressure_lag_6,output_flow_rate_lag_6,pump_1_lag_6,pump_2_lag_6,input_flow_rate_lag_12,reservoir_level_percentage_lag_12,pressure_lag_12,output_flow_rate_lag_12,pump_1_lag_12,pump_2_lag_12,input_flow_rate_lag_24,reservoir_level_percentage_lag_24,pressure_lag_24,output_flow_rate_lag_24,pump_1_lag_24,pump_2_lag_24,air_temp_c_lag_1,total_precip_mm_lag_1,relative_humidity_percentage_lag_1,air_temp_c_lag_2,total_precip_mm_lag_2,relative_humidity_percentage_lag_2,air_temp_c_lag_3,total_precip_mm_lag_3,relative_humidity_percentage_lag_3,air_temp_c_lag_6,total_precip_mm_lag_6,relative_humidity_percentage_lag_6,air_temp_c_lag_12,total_precip_mm_lag_12,relative_humidity_percentage_lag_12,air_temp_c_lag_24,total_precip_mm_lag_24,relative_humidity_percentage_lag_24,input_flow_rate_roll_mean_3,reservoir_level_percentage_roll_mean_3,pressure_roll_mean_3,output_flow_rate_roll_mean_3,pump_1_roll_mean_3,pump_2_roll_mean_3,input_flow_rate_roll_mean_6,reservoir_level_percentage_roll_mean_6,pressure_roll_mean_6,output_flow_rate_roll_mean_6,pump_1_roll_mean_6,pump_2_roll_mean_6,input_flow_rate_roll_mean_12,reservoir_level_percentage_roll_mean_12,pressure_roll_mean_12,output_flow_rate_roll_mean_12,pump_1_roll_mean_12,pump_2_roll_mean_12,input_flow_rate_roll_mean_24,reservoir_level_percentage_roll_mean_24,pressure_roll_mean_24,output_flow_rate_roll_mean_24,pump_1_roll_mean_24,pump_2_roll_mean_24,air_temp_c_roll_mean_3,total_precip_mm_roll_mean_3,relative_humidity_percentage_roll_mean_3,air_temp_c_roll_mean_6,total_precip_mm_roll_mean_6,relative_humidity_percentage_roll_mean_6,air_temp_c_roll_mean_12,total_precip_mm_roll_mean_12,relative_humidity_percentage_roll_mean_12,air_temp_c_roll_mean_24,total_precip_mm_roll_mean_24,relative_humidity_percentage_roll_mean_24,hour,day_of_week,week_of_year,year,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24
4350,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,3.16,92.26,32.62,0.0,0.0,0.0,8.8,0.0,87.0,8.8,0.0,87.0,8.8,0.0,87.0,8.8,0.0,87.0,8.8,0.0,87.0,18.6,0.0,84.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,0.0,47.8,32.36,1.77,0.0,0.0,1.316667,66.325,32.468333,1.16375,0.0,0.0,8.8,0.0,87.0,8.8,0.0,87.0,8.8,0.0,87.0,12.883333,0.0,85.75,5,4,37,2023,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,1.77,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06,73.06


In [41]:
len(df2.columns)

118